## NA imputation 

In [1]:
import jusipy
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import missingno as msno
import copy
from fast_imputation import fast_imputation

#### Loading...

In [2]:
all_points = pd.read_pickle('data/all_points.pkl')

In [3]:
all_points.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22107 entries, 0 to 19999
Data columns (total 6 columns):
lat         22107 non-null object
long        22107 non-null float64
country     22107 non-null object
year        22107 non-null object
positive    22107 non-null int64
random      22107 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ MB


In [4]:
A_country = jusipy.country_features.All()

Loading WorldBank(/home/marios/Desktop/Personal/repos/JuSiPy_Asser/jusipy/country_features/data/SD_API_15_DS2_en_csv_v2_10186395.csv)                                                                                                    

In [5]:
country_features = jusipy.country_features.get(all_points, A_country, fuzzy=50)

In [6]:
all_points2 = all_points.drop_duplicates(subset=['country','year'])

In [7]:
all_points2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776 entries, 0 to 19844
Data columns (total 6 columns):
lat         776 non-null object
long        776 non-null float64
country     776 non-null object
year        776 non-null object
positive    776 non-null int64
random      776 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 42.4+ KB


In [8]:
country_features = jusipy.country_features.get(all_points2, A_country, fuzzy=50)

In [9]:
country_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776 entries, 0 to 19844
Columns: 396 entries, WB_LGAF__WB_LGAF2016_1_1_1 to WB_SD__SP_DYN_LE00_MA_IN
dtypes: float64(280), object(116)
memory usage: 2.4+ MB


-------------------------------------------------------------------------------

### NA percentage (in red)

In [10]:
# somehow unconsciouly it came out like the French flag..

In [27]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS']
fields_3 = list(country_features.keys())
#for x in fields_3:
#    df = country_features[x]
#    if x in fields:
#        print('\033[94m'+x+': '+'\033[92m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
#    else:
#        print('\033[94m'+x+': '+'\033[0m'+A_country.description[x]+' has '+str(sum(df.isna()))+' NA '+'\033[91m'+str(sum(df.isna())/len(df)))
nas = []
nas_per = []
des = []
for x in fields_3:
    df = country_features[x]
    nas.append(sum(df.isna()))
    nas_per.append(sum(df.isna())/len(df))
    des.append(A_country.description[x])

    ###Printing the ones with NA < 10%
DF = pd.DataFrame({'Indicator': fields_3, 'Description': des, 'Percentage of NA':nas_per})
DF[DF["Percentage of NA"]<0.1]

,Indicator,Description,Percentage of NA
116,WB_SE__WB_NY_GDP_MKTP_KD_ZG,GDP growth (annual %),0.036082
117,WB_SE__WB_NY_GDP_MKTP_PP_KD,"GDP, PPP (constant 2011 international $)",0.061856
158,TI_CPI__TI_CPI_CPIRN,Corruption Perception Index - Country Ranking ...,0.020619
159,TI_CPI__TI_CPI_CPIRO,Corruption Perception Index - Country Ranking ...,0.024485
160,TI_CPI__TI_CPI_CPISN,Corruption Perception Index - Score (New Metho...,0.020619
161,TI_CPI__TI_CPI_CPISO,Corruption Perception Index - Score (Old Metho...,0.024485
162,WB_ARD__AG_AGR_TRAC_NO,"Agricultural machinery, tractors",0.070876
164,WB_ARD__AG_CON_FERT_ZS,Fertilizer consumption (kilograms per hectare ...,0.092784
165,WB_ARD__AG_LND_AGRI_K2,Agricultural land (sq. km),0.014175
166,WB_ARD__AG_LND_AGRI_ZS,Agricultural land (% of land area),0.021907


In [12]:
fields = ['WB_ARD__AG_SRF_TOTL_K2', 'WB_SE__WB_NY_GDP_MKTP_KD_ZG', 'WB_ARD__SP_RUR_TOTL', 'WB_ARD__EG_ELC_ACCS_RU_ZS', 'WB_ARD__AG_LND_TOTL_RU_K2',
          'WB_ARD__AG_LND_FRST_K2', 'WB_ARD__AG_LND_ARBL_ZS', 'WB_ARD__AG_LND_AGRI_K2', 'WB_ARD__AG_CON_FERT_ZS','WB_ARD__AG_LND_PRCP_MM','WB_ARD__EN_ATM_METH_AG_KT_CE',
         'WB_ARD__EN_ATM_NOXE_AG_KT_CE','WB_ARD__NV_AGR_TOTL_ZS','WB_ARD__TM_VAL_AGRI_ZS_UN','WB_ARD__TX_VAL_AGRI_ZS_UN','WB_ARD__AG_LND_TOTL_K2',
         'WB_ARD__SL_AGR_EMPL_ZS']
imputed_ = []
for x in fields:
    df_temp = country_features[x]
    imputed_.append(fast_imputation(df_temp))

In [13]:
all_points_new = copy.deepcopy(all_points)

#### New dataset

In [14]:
for i in range(len(fields)):
    x = fields[i]
    all_points_new[x] = imputed_[i]

all_points_new.to_pickle('./data/all_points_new.pkl')

In [15]:
all_points_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22107 entries, 0 to 19999
Data columns (total 23 columns):
lat                             22107 non-null object
long                            22107 non-null float64
country                         22107 non-null object
year                            22107 non-null object
positive                        22107 non-null int64
random                          22107 non-null int64
WB_ARD__AG_SRF_TOTL_K2          1525 non-null float64
WB_SE__WB_NY_GDP_MKTP_KD_ZG     1525 non-null float64
WB_ARD__SP_RUR_TOTL             1525 non-null float64
WB_ARD__EG_ELC_ACCS_RU_ZS       1525 non-null float64
WB_ARD__AG_LND_TOTL_RU_K2       1525 non-null float64
WB_ARD__AG_LND_FRST_K2          1525 non-null float64
WB_ARD__AG_LND_ARBL_ZS          1525 non-null float64
WB_ARD__AG_LND_AGRI_K2          1525 non-null float64
WB_ARD__AG_CON_FERT_ZS          1525 non-null float64
WB_ARD__AG_LND_PRCP_MM          1525 non-null float64
WB_ARD__EN_ATM_METH_AG_KT_CE

#### The columns added are (in order):

In [16]:
for x in fields:
    print(A_country.description[x])

Surface area (sq. km)
GDP growth (annual %)
Rural population
Access to electricity, rural (% of rural population)
Rural land area (sq. km)
Forest area (sq. km)
Arable land (% of land area)
Agricultural land (sq. km)
Fertilizer consumption (kilograms per hectare of arable land)
Average precipitation in depth (mm per year)
Agricultural methane emissions (thousand metric tons of CO2 equivalent)
Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent)
Agriculture, forestry, and fishing, value added (% of GDP)
Agricultural raw materials imports (% of merchandise imports)
Agricultural raw materials exports (% of merchandise exports)
Land area (sq. km)
Employment in agriculture (% of total employment) (modeled ILO estimate)


#### Also this paper https://academic.oup.com/bioinformatics/article/21/8/1509/249540, suggests that for highly correlated features the optimal feature size is
#### sqrt(N), where N is the sample size. For us this would be roughly 116 features.. 

In [18]:
gdp = country_features['WB_SE__WB_NY_GDP_MKTP_KD_ZG']

4.834865000000001